In [1]:
import os
import math
import tensorflow as tf
import datetime

/home/hhl/anaconda3/lib/python3.6/site-packages/h5py/__init__.py:34: FutureWarning: Conversion of the second argument of issubdtype from `float` to `np.floating` is deprecated. In future, it will be treated as `np.float64 == np.dtype(float).type`.
  from ._conv import register_converters as _register_converters


## 数据预处理--处理好每个pin的数据。 

In [2]:
pin2value = dict()

In [3]:
with open('data/sdtm_final.txt','r') as f:
    for line in f:
        line = line.strip().split('\t')
        pin = line[0]
        data = line[1]
        dt = line[2]
        if pin in pin2value:
            dt2data = pin2value.get(pin)
        else:
            dt2data = {}
        dt2data[dt] = data
        pin2value[pin] = dt2data

In [4]:
def get_date_list(datestart = None,dateend = None):
    # 创建日期辅助表
    if datestart is None:
        datestart = '20170101'
    if dateend is None:
        dateend = datetime.datetime.now().strftime('%Y%m%d')

    # 转为日期格式
    datestart=datetime.datetime.strptime(datestart,'%Y%m%d')
    dateend=datetime.datetime.strptime(dateend,'%Y%m%d')
    date_list = []
    date_list.append(datestart.strftime('%Y%m%d'))
    while datestart<dateend:
        # 日期叠加一天
        datestart+=datetime.timedelta(days=+1)
        # 日期转字符串存入列表
        date_list.append(datestart.strftime('%Y%m%d'))
    return date_list
#     print(date_list)

In [5]:
date_list = get_date_list('2018061','20181130')

In [6]:
pin2data_final = {}
for pin,value in pin2value.items():
    data_sortedby_dt = []
    for date in date_list:
        if date in value:
            data_sortedby_dt.append(value[date].split('|'))
        else:
            week = datetime.datetime.strptime(date,'%Y%m%d').weekday()
            strvalue = '0|0|0|0|0|0|0|0|0|0|0|0|0|0|0|0|0|0|0|0|0|0|0|0|'+str(week)+'|0'
            data_sortedby_dt.append(strvalue.split('|'))
    if '20181130' in value:
        pin2data_final[pin] = data_sortedby_dt

In [7]:
len(pin2data_final)

69281

In [8]:
len(pin2value)

499435

In [18]:
i = 0 
with open('data/pin2data_final.txt','w') as f:
    for pin,value in pin2data_final.items():
        i += 1
        if i > 10 :
            break
        print(len(value[0:-1][0]))
        print(len(value[-1][8:20]))
        f.write(str(value[0:-1]) + '\t' + str(value[-1][8:21]) +'\n')

26
13
26
13
26
13
26
13
26
13
26
13
26
13
26
13
26
13
26
13


In [17]:
def get_batch(data, batch_size):
    x_batch,y_batch = [], []
    for pin, data_sortedby_dt in data.items():
        if len(x_batch) == batch_size :
            yield x_batch, y_batch
            x_batch, y_batch = [], []
        x_batch += data_sortedby_dt[:-1]
        y_batch += data_sortedby_dt[-1][8:21]

In [18]:
batch_size = 32

In [20]:
for i, (x_batch, y_batch) in enumerate(get_batch(pin2data_final, batch_size)):
    print(i)